## Columns to be removed from training/validation

# Load Tensorflow and check GPU availability

In [1]:
# -*- coding: utf-8 -*-
import torch
from core.loader import Loader
from models.model_wrapper import ModelWrapper


from tensorflow.python.client import device_lib

for device in device_lib.list_local_devices():
    print(device.physical_device_desc)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


2025-05-02 22:29:12.380006: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-02 22:29:12.380036: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-02 22:29:12.381340: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-02 22:29:12.387634: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-02 22:29:13.091512: W tensorflow/compiler/tf2


device: 0, name: NVIDIA GeForce RTX 3050 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


2025-05-02 22:29:13.864802: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-02 22:29:13.904248: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-02 22:29:13.907051: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

# Load input datasets

In [ ]:
benign_dataset_filenames = [
    'parkets/benign/benign_2312_anonymized_HTML.parquet', 
    'parkets/benign/umbrella_benign_FINISHED_HTML.parquet'
        
]
malicious_dataset_filenames = [
    'parkets/malware_2406_strict_HTML.parquet'
]


# print me number of domains from each separate dataset

# CONFIGURATION

benign_label = "benign"
malicious_label = "phishing"

class_map = {benign_label: 0, malicious_label: 1}
# print labels from malicious datasets

loader = Loader(benign_dataset_filenames, malicious_dataset_filenames, benign_label=benign_label, malicious_label=malicious_label, subsample=1.0)
df = loader.load()

# split into 3 stages 

In [3]:
from core.loader import Segmenter

# Define the aggregates that needs to be created

aggregates = [
    ["lex_"],
    ["lex_", "dns_", "ip_", "geo_"],
    ["lex_", "dns_", "ip_", "tls_", "geo_", "rdap_"],
]

segmenter = Segmenter(df)
segmenter.create_base_subsets() # create base subsets
segmenter.create_aggregated_subsets(aggregates)
subset_dfs = segmenter.get_aggregated_subsets()

/home/poli/Desktop/git/deep_domain_detection/src/core/loader.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df["label"] = self.df["label"].copy()
/home/poli/Desktop/git/deep_domain_detection/src/core/loader.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df["label"] = self.df["label"].copy()
/home/poli/Desktop/git/deep_domain_detection/src/core/loader.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

# Data preprocessing

In [ ]:
labels = df['label'].apply(lambda x: class_map[x]) # y vector
features = df.drop('label', axis=1).copy() # X matrix

features = loader.scale(features)

# Train-test split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
  features,
  labels,
  test_size=0.2,
  random_state=42,
  shuffle=True, 
  stratify=labels
)


# Define the NN model

In [4]:

from tensorflow.keras.models import Model
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam

# TODO: Testovat NN na prvni 3 stupne. 
# TODO: Pro každý stupeň vybrat nejlepší model

# TODO: Měřit na celém datasetu + potom udělat validaci na CLF testu


ARCH_NAME = "feedforward"
VERSION = "v1.0"
LR = 0.0023

def build_feedforward_net(feature_size):
    # Input layer
    inputs = Input(shape=(feature_size,))
    
    # First hidden layer
    
    x = Dense(1024, activation=None)(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # Second hidden layer
    x = Dense(512, activation=None)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # Third hidden layer
    x = Dense(256, activation=None)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # Fourth hidden layer
    x = Dense(128, activation=None)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Output layer
    outputs = Dense(1, activation='sigmoid')(x)

    # Build and return the model
    model = Model(inputs=inputs, outputs=outputs, name="ARCH_NAME")
    return model



In [ ]:
# save subset_dfs to pkl file to tmp folder
import os
import tempfile
import pickle
import torch
from core.loader import Loader
from models.model_wrapper import ModelWrapper
from tensorflow.python.client import device_lib

for device in device_lib.list_local_devices():
    print(device.physical_device_desc)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

save_file = "subset_dfs_malware.pkl"
tmp_dir_name = "tmp"


tmp_dir = os.path.join(os.getcwd(), tmp_dir_name)
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)
tmp_file = os.path.join(tmp_dir, save_file)

def save_subset_dfsp(subset_dfs):
    with open(tmp_file, 'wb') as f:
        pickle.dump(subset_dfs, f)
    return tmp_file

def load_subset_dfs_from_tmp(tmp_file):
    with open(tmp_file, 'rb') as f:
        subset_dfs = pickle.load(f)
    return subset_dfs

### Save subsets

In [ ]:
# save subsets
save_subset_dfsp(subset_dfs)

### Load subsets

In [ ]:
# load subsets 
subset_dfs = load_subset_dfs_from_tmp(tmp_file)

In [ ]:
import gc
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from models.model_wrapper import ModelWrapper


model_histories = []

# make sure TF only allocates as much GPU memory as it needs
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

wrapper = ModelWrapper()

i = 1
for prefix, subset_df in subset_dfs.items():
    if i == 3:
        print("We already have stage 3")
        break

    print(f"\n🚀 Training Feedforward NN on '{prefix}' features…")
    labels   = subset_df['label'].map(class_map)
    features = loader.scale(subset_df.drop('label', axis=1), stage=i, model=ARCH_NAME)

    X_train, X_test, Y_train, Y_test = train_test_split(
        features, labels,
        test_size=0.2, random_state=42,
        shuffle=True, stratify=labels
    )

    model = build_feedforward_net(X_train.shape[1])
    model.compile(
        optimizer=Adam(learning_rate=LR),
        loss='binary_crossentropy',
        metrics=['Precision', 'Recall', 'AUC']
    )

    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )

    history = model.fit(
        X_train, Y_train,
        batch_size=512,
        epochs=15,
        validation_data=(X_test, Y_test),
        class_weight={0: 1.0, 1: 0.8},
        callbacks=[early_stopping]
    )
    

    model_histories.append({"model_name": prefix,
                      "history": history})
    

    wrapper.save(model,
                 arch_name=ARCH_NAME,
                 label=malicious_label,
                 prefix=prefix,
                 version=VERSION)

    # ---- here’s the magic ----
    K.clear_session()    # drops the entire TF graph + variables
    del model           # remove the Python reference
    del history         # free training history
    del X_train, X_test, Y_train, Y_test, features, labels
    gc.collect()        # ask Python to free unreferenced memory
    i += 1

2025-05-02 22:32:03.726051: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-02 22:32:03.733782: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-02 22:32:03.736679: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-


🚀 Training Feedforward NN on 'lex_agg' features…


2025-05-02 22:32:33.429815: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-02 22:32:33.433003: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-02 22:32:33.435792: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Epoch 1/25


2025-05-02 22:33:10.155717: I external/local_xla/xla/service/service.cc:168] XLA service 0x7780d4017ae0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-05-02 22:33:10.155741: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2025-05-02 22:33:10.175511: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-05-02 22:33:10.333913: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
I0000 00:00:1746217990.419957 1131486 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1434/1434 [==============================] - 39s 6ms/step - loss: 0.2223 - precision: 0.7019 - recall: 0.3401 - auc: 0.8931 - val_loss: 0.2176 - val_precision: 0.6850 - val_recall: 0.4001 - val_auc: 0.8983
Epoch 2/25
1434/1434 [==============================] - 7s 5ms/step - loss: 0.2051 - precision: 0.7331 - recall: 0.3937 - auc: 0.9110 - val_loss: 0.2027 - val_precision: 0.7241 - val_recall: 0.4285 - val_auc: 0.9135
Epoch 3/25
1434/1434 [==============================] - 7s 5ms/step - loss: 0.1993 - precision: 0.7412 - recall: 0.4094 - auc: 0.9166 - val_loss: 0.1987 - val_precision: 0.7618 - val_recall: 0.3880 - val_auc: 0.9172
Epoch 4/25
1434/1434 [==============================] - 7s 5ms/step - loss: 0.1961 - precision: 0.7458 - recall: 0.4183 - auc: 0.9198 - val_loss: 0.1975 - val_precision: 0.7155 - val_recall: 0.4563 - val_auc: 0.9185
Epoch 5/25
1434/1434 [==============================] - 7s 5ms/step - loss: 0.1928 - precision: 0.7520 - recall: 0.4284 - auc: 0.9228 - val_loss: 

2025-05-02 22:37:10.800631: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 751314944 exceeds 10% of free system memory.
2025-05-02 22:37:11.583246: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 751314944 exceeds 10% of free system memory.


Epoch 1/25
1434/1434 [==============================] - ETA: 0s - loss: 0.1080 - precision: 0.8245 - recall: 0.7629 - auc: 0.9786

2025-05-02 22:37:23.292705: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 187829248 exceeds 10% of free system memory.


1434/1434 [==============================] - 12s 7ms/step - loss: 0.1080 - precision: 0.8245 - recall: 0.7629 - auc: 0.9786 - val_loss: 0.0928 - val_precision: 0.8871 - val_recall: 0.7531 - val_auc: 0.9839
Epoch 2/25
1434/1434 [==============================] - 7s 5ms/step - loss: 0.0857 - precision: 0.8611 - recall: 0.8092 - auc: 0.9863 - val_loss: 0.0880 - val_precision: 0.8582 - val_recall: 0.8042 - val_auc: 0.9851
Epoch 3/25
1434/1434 [==============================] - 7s 5ms/step - loss: 0.0803 - precision: 0.8685 - recall: 0.8239 - auc: 0.9879 - val_loss: 0.0796 - val_precision: 0.8565 - val_recall: 0.8448 - val_auc: 0.9871
Epoch 4/25
1434/1434 [==============================] - 7s 5ms/step - loss: 0.0738 - precision: 0.8792 - recall: 0.8370 - auc: 0.9896 - val_loss: 0.0761 - val_precision: 0.8641 - val_recall: 0.8531 - val_auc: 0.9892
Epoch 5/25
1434/1434 [==============================] - 8s 5ms/step - loss: 0.0703 - precision: 0.8839 - recall: 0.8439 - auc: 0.9906 - val_loss: 

In [ ]:
def get_metric(history, metric_name):
    if metric_name in history.history:
        return history.history[metric_name]
    elif f"{metric_name}_12" in history.history:
        return history.history[f"{metric_name}_12"]
    elif f"{metric_name}_2" in history.history:
        return history.history[f"{metric_name}_2"]
    else:
        raise KeyError(f"Metric {metric_name} not found in history.")

epoch_losses = get_metric(history, 'loss')
epoch_val_losses = get_metric(history, 'val_loss')
epoch_accuracies = get_metric(history, 'auc')
epoch_val_accuracies = get_metric(history, 'val_auc')
epoch_precisions = get_metric(history, 'precision')
epoch_val_precisions = get_metric(history, 'val_precision')
epoch_recalls = get_metric(history, 'recall')
epoch_val_recalls = get_metric(history, 'val_recall')

# Calculate F1 score
epoch_f1s = [2 * (p * r) / (p + r) for p, r in zip(epoch_precisions, epoch_recalls)]
epoch_val_f1s = [2 * (p * r) / (p + r) for p, r in zip(epoch_val_precisions, epoch_val_recalls)]

import matplotlib.pyplot as plt

plt.figure(figsize=(18, 10))

# Plot for Loss
plt.subplot(2, 3, 1)
plt.plot(epoch_losses, linestyle='--', marker='o', color='b', label='Training Loss')
plt.plot(epoch_val_losses, linestyle='--', marker='o', color='r', label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss')
plt.legend()
plt.grid(True)

# Plot for AUC (as a proxy for Accuracy)
plt.subplot(2, 3, 2)
plt.plot(epoch_accuracies, linestyle='--', marker='o', color='#ff7f0e', label='Training AUC')
plt.plot(epoch_val_accuracies, linestyle='--', marker='o', color='r', label='Validation AUC')
plt.xlabel('Epoch')
plt.ylabel('AUC')
plt.title('AUC (Accuracy)')
plt.legend()
plt.grid(True)

# Plot for Precision
plt.subplot(2, 3, 3)
plt.plot(epoch_precisions, linestyle='--', marker='o', color='g', label='Training Precision')
plt.plot(epoch_val_precisions, linestyle='--', marker='o', color='r', label='Validation Precision')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.title('Precision')
plt.legend()
plt.grid(True)

# Plot for Recall
plt.subplot(2, 3, 4)
plt.plot(epoch_recalls, linestyle='--', marker='o', color='c', label='Training Recall')
plt.plot(epoch_val_recalls, linestyle='--', marker='o', color='r', label='Validation Recall')
plt.xlabel('Epoch')
plt.ylabel('Recall')
plt.title('Recall')
plt.legend()
plt.grid(True)

# Plot for F1 Score
plt.subplot(2, 3, 5)
plt.plot(epoch_f1s, linestyle='--', marker='o', color='m', label='Training F1 Score')
plt.plot(epoch_val_f1s, linestyle='--', marker='o', color='r', label='Validation F1 Score')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.title('F1 Score')
plt.legend()
plt.grid(True)

plt.suptitle('NN Training Progress', fontsize=16, y=1.02)
plt.tight_layout(rect=[0, 0, 1, 0.96])



# Save the figure
plt.savefig(f'./figures/training_{ARCH_NAME}_{malicious_label}_{VERSION}.png', dpi=500, bbox_inches='tight', pad_inches=0.5)
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

# Evaluate the model
loss_and_metrics = model.evaluate(X_test, Y_test)
print('Loss = ', loss_and_metrics[0])
print('Accuracy = ', loss_and_metrics[1])

# Generate predictions
Y_pred = model.predict(X_test)
Y_pred = np.round(Y_pred).astype(int)  # Convert probabilities to binary predictions

# Calculate additional metrics
precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred)

# Confusion matrix
cm = confusion_matrix(Y_test, Y_pred)

# False Positive Rate
tn, fp, fn, tp = cm.ravel()
fpr = fp / (fp + tn)

# Display the metrics
print('\n=== RESULTS ===')
print(classification_report(Y_test, Y_pred, target_names=['Benign', 'Malicious'], digits=4))
print('False Positive Rate =', fpr)


# Display the confusion matrix
print('\nConfusion Matrix:')
print(cm)

# Optionally, plot the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
fig, ax = plt.subplots(figsize=(7, 7))  # Increase figure size for better readability
disp.plot(ax=ax, values_format='d')
for labels in disp.text_:
    for label in labels:
        label.set_fontsize(18) 
plt.show()

# TODO: ADD clf test

In [ ]:
analyze_feature_importance = True

if analyze_feature_importance:
    
    import shap
    
    # Ensure that X_train and X_test are DataFrames with the correct column names
    # You can set the column names from the 'features' DataFrame like this:
    X_train.columns = features.columns
    X_test.columns = features.columns
    
    n_samples = 1000
    
    # Convert your training set to a NumPy format if it's not already
    background = X_train[:n_samples].to_numpy()
    
    # Use the generic SHAP Explainer interface
    explainer = shap.Explainer(model, background)
    
    # Generate SHAP values for the test set
    shap_values = explainer(X_test[:n_samples].to_numpy(), max_evals=1000)
    
    # Plotting the summary plot for feature importance
    # Use the column names from the 'features' DataFrame as the feature names
    shap.summary_plot(shap_values.values, X_test[:n_samples], feature_names=features.columns, max_display=30)
